# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [12]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Check basic VGG19 Classification

In [13]:
testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# Convert Image pixels to a numpy array to be operated on
pictureArray = img_to_array(testPicture)
# reshape Data for model
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
# prepare the image for the VGG model
pictureArray = preprocess_input(pictureArray)
# predict the probability across all output classes
yhat = preTrainedModel.predict(pictureArray)
# convert the probabilities to class labels
labels = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = labels[0][0]
# print the classification
print(labels)



1/1 [==============================] - 0s 176ms/step
[[('n02441942', 'weasel', 0.3109628), ('n02443484', 'black-footed_ferret', 0.17763011), ('n02443114', 'polecat', 0.17633283), ('n02447366', 'badger', 0.16493277), ('n02442845', 'mink', 0.059160564)]]


2023-10-29 15:01:48.430606: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


## Create complete Model with Custom output Layer

In [14]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
# completeModel.add(Dense(75, activation = "relu"))
# completeModel.add(Dense(50, activation = "relu"))
# completeModel.add(Dense(25, activation = "relu"))
completeModel.add(Dense(1, activation = "sigmoid"))
completeModel.summary()
completeModel.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense_4 (Dense)             (None, 100)               100100    
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 143,767,441
Trainable params: 100,201
Non-trainable params: 143,667,240
_________________________________________________________________


/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Orgainze test and training data
Be sure to change file contents

In [15]:
trainingValues = [0] * 895 + [1] * 1070
testValues = [1] * 160 + [0] * 154
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels = trainingValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))
testImages = keras.utils.image_dataset_from_directory(
    directory='testData',
    labels= testValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))
print (trainImages)

Found 1965 files belonging to 2 classes.
Found 314 files belonging to 2 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>


## Prediction before training

In [16]:
testPicture = load_img('../rabbitNightCrop1.jpg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-10-29 15:01:48.788756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 182ms/step
[[0.5005997]]


## Train Last two layers

In [17]:
completeModel.fit(trainImages, epochs=10, batch_size=100)

Epoch 1/10


2023-10-29 15:01:49.402461: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


62/62 [==============================] - 21s 319ms/step - loss: 0.6369 - accuracy: 0.7374
Epoch 2/10
62/62 [==============================] - 20s 313ms/step - loss: 0.4874 - accuracy: 0.8433
Epoch 3/10
62/62 [==============================] - 20s 312ms/step - loss: 0.3706 - accuracy: 0.8728
Epoch 4/10
62/62 [==============================] - 20s 312ms/step - loss: 0.3138 - accuracy: 0.8845
Epoch 5/10
62/62 [==============================] - 20s 312ms/step - loss: 0.2811 - accuracy: 0.8982
Epoch 6/10
62/62 [==============================] - 20s 314ms/step - loss: 0.2587 - accuracy: 0.9074
Epoch 7/10
62/62 [==============================] - 20s 314ms/step - loss: 0.2424 - accuracy: 0.9176
Epoch 8/10
62/62 [==============================] - 19s 312ms/step - loss: 0.2279 - accuracy: 0.9237
Epoch 9/10
62/62 [==============================] - 20s 313ms/step - loss: 0.2175 - accuracy: 0.9257
Epoch 10/10
62/62 [==============================] - 20s 313ms/step - loss: 0.2077 - accuracy: 0.9318


## Test Some data

In [18]:
score = completeModel.evaluate(testImages)
print ("loss = ", score[0])
print ("accuracy = ", score[1])

2023-10-29 15:05:06.186902: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 4s 353ms/step - loss: 1.8516 - accuracy: 0.3344
loss =  1.851609706878662
accuracy =  0.3343949019908905


## See what happens with a deer image

In [19]:
from PIL import Image
testPicture = load_img('testData/testDeer/SEQ86955_IMG_0004.JPG___crop00_md_v5a.0.0.pt.jpg', target_size=(224,224))
# testPicture = load_img('../deerNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../foxNightCrop2.jpg', target_size=(224,224))
# testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../downloadRabbit.jpeg', target_size=(224,224))
# testPicture = load_img('../rabbitNightCrop1.jpg', target_size=(224,224))
testPicture.show()
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

1/1 [==============================] - 0s 42ms/step
[[0.8351684]]


## Save the model for export

In [20]:
completeModel.save("rabbitModel.keras")

## Check test data for each animal

In [21]:
# import required module
import os
# assign directory
directory = 'testData/testDeer'
 
# iterate over files in
# that directory
results = [0,0]
# 0 is positives 1 is negatives
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1

print (results)

testData/testDeer/SEQ81957_IMG_0003.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.32269195]]
testData/testDeer/SEQ80916_IMG_0004.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.8049543]]
testData/testDeer/SEQ86955_IMG_0001.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.01639291]]
testData/testDeer/SEQ87558_IMG_0003.JPG___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.90486217]]
testData/testDeer/SEQ81957_IMG_0010.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.5311009]]
testData/testDeer/SEQ86931_IMG_0003.JPG___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.50235623]]
testData/testDeer/SEQ81957_IMG_0016.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.46809915]]
testData/testDeer/SEQ87611_IMG_0006.JPG___crop01_md_v5a.0.0.pt.j

In [23]:
directory = 'extraTest/testFox'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testFox/SEQ81302_IMG_0004.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.9941825]]
extraTest/testFox/SEQ81302_IMG_0004.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.9921799]]
extraTest/testFox/SEQ81296_IMG_0004.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.8341252]]
extraTest/testFox/SEQ80191_IMG_0010.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.01528856]]
extraTest/testFox/SEQ80191_IMG_0009.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.02544037]]
extraTest/testFox/SEQ

In [24]:
directory = 'extraTest/testPossum'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testPossum/1980.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.2978426]]
extraTest/testPossum/2005.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9782591]]
extraTest/testPossum/2016.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.7484343]]
extraTest/testPossum/2059.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.06167734]]
extraTest/testPossum/1993.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9170448]]
extraTest/testPossum/2123.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.14633694]]
extraTest/testPossum/1978.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.1732253]]
extraTest/testPossum/2130.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.32043374]]
extraTest/tes

In [25]:
directory = 'extraTest/testRabbit'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testRabbit/791.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.70203817]]
extraTest/testRabbit/782.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.67610407]]
extraTest/testRabbit/735.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9344898]]
extraTest/testRabbit/821.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.7209439]]
extraTest/testRabbit/769.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.7771309]]
extraTest/testRabbit/832.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.7513059]]
extraTest/testRabbit/726.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.94767994]]
extraTest/testRabbit/784.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.8936821]]
extraTest/testRabbit/

In [26]:
directory = 'extraTest/testSquirrel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testSquirrel/85.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 16ms/step
[[0.25254413]]
extraTest/testSquirrel/83.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01331909]]
extraTest/testSquirrel/89.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01118983]]
extraTest/testSquirrel/97.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.05738038]]
extraTest/testSquirrel/91.jpg___crop04_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.00729803]]
extraTest/testSquirrel/96.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.00142117]]
extraTest/testSquirrel/82.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.15884753]]
extraTest/testSquirrel/84.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.00570115]]
extraTes

## Code for data manipulation to creaet more data

In [ ]:
# directory = 'trainingData/trainRabbit' #Just change directory name for different animal
 
# # iterate over files in
# # that directory
# results = [0,0,0,0]
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         testPicture = load_img(f, target_size=(224,224))
#         newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
#         newPic.save(f[:-4] + "FlippedImage" + f[-4:])